# PALABRIA – Notebook de ejecución

Este notebook permite ejecutar la aplicación PALABRIA de forma sencilla en Google Colab, y exponerlo mediante una URL pública.


Privacidad y datos:
- Cada usuario crea su propia base de datos.
- La base de datos NO se comparte con los autores del proyecto.


In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# Ejecuta esta celda SOLO si ya tienes la carpeta aplicacion_PALABRIA en tu Drive

import os, sys

PROJECT_PATH = "/content/drive/MyDrive/aplicacion_PALABRIA"

if not os.path.exists(PROJECT_PATH):
    raise FileNotFoundError(
        "No se encontró la carpeta 'aplicacion_PALABRIA' en tu Drive.\n"
        "Copia ahí el proyecto o usa la celda de GitHub."
    )

os.chdir(PROJECT_PATH)
sys.path.append(PROJECT_PATH)

In [21]:
# Ejecuta esta celda SOLO si vas a descargar el proyecto desde GitHub en el sistema TEMPORAL de Colab

import os, sys

PROJECT_PATH = "/content/drive/MyDrive/aplicacion_PALABRIA"
REPO_URL = "https://github.com/ncenteno-arch/APP-PALABRIA.git"


if not os.path.exists(PROJECT_PATH):
    !git clone $REPO_URL $PROJECT_PATH

os.chdir(PROJECT_PATH)
sys.path.append(PROJECT_PATH)

In [22]:
# Define la base de datos SQL
# La BD se guarda en tú Google Drive, independientemente de si el código viene de Drive o de GitHub

import pathlib

DB_PATH = "/content/drive/MyDrive/PALABRIA/data/palabria.db"
pathlib.Path(os.path.dirname(DB_PATH)).mkdir(parents=True, exist_ok=True)

os.environ["DB_PATH"] = DB_PATH

print("Base de datos personal configurada en:", DB_PATH)

Base de datos personal configurada en: /content/drive/MyDrive/PALABRIA/data/palabria.db


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.7 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=4f0c88f4eafc66c7728c9acf66815490e276926f66d8406a026316187e5ebd72
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d7

In [5]:
# Se descarga el modelo lingüístico de spaCy para español (necesario para la app)

!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [23]:
# Permite que el backend pueda quedarse funcionando en segundo plano, de forma que después podamos arrancar la interfaz web (frontend) sin que el notebook se bloquee

import nest_asyncio, logging

nest_asyncio.apply()
logging.basicConfig(level=logging.INFO)

In [24]:
# Se lanza el backend en local

import subprocess, time, os

os.chdir(PROJECT_PATH)

backend_process = subprocess.Popen(
    [
        "uvicorn",
        "backend.main:app",
        "--host", "127.0.0.1",
        "--port", "8000",
        "--log-level", "warning"
    ]
)

time.sleep(15)
print("Backend activo")

Backend activo


In [25]:
# El frontend se conecta al backend local

import os

BACKEND_URL = "http://127.0.0.1:8000"

os.environ["BACKEND_URL"] = BACKEND_URL

print("Frontend conectado al backend")

Frontend conectado al backend


In [26]:
# Introduce TU token de ngrok (ngrok crea una dirección web pública temporal)

from pyngrok import ngrok

NGROK_TOKEN = input("Pega aquí tu NGROK AUTHTOKEN (no se guarda): ").strip()
ngrok.set_auth_token(NGROK_TOKEN)
ngrok.kill()

Pega aquí tu NGROK AUTHTOKEN (no se guarda): 32pBH53FrH3RTAqrYXzXqtYTpZp_7Y6ZsXYkb7dF1i1ddHCF7


In [27]:
# Se lanza el frontend Streamlit y se muestra la URL visible externamente con ngrok

import subprocess

frontend_tunnel = ngrok.connect(8501)

subprocess.Popen([
    "streamlit", "run", "frontend/app.py",
    "--server.port=8501",
    "--server.headless=true",
    "--browser.serverAddress=0.0.0.0"
])

print("APLICACIÓN DISPONIBLE EN:")
print(frontend_tunnel.public_url)

APLICACIÓN DISPONIBLE EN:
https://9db1e3c95964.ngrok-free.app


**Recomendación:** lo más fiable es reiniciar el entorno de Colab para volver a ejecutar la aplicación.
Si no quieres reiniciar, prueba primero a limpiar los procesos en segundo plano.

In [28]:
# Limpia los procesos anteriores (backend, frontend y ngrok)

import subprocess

subprocess.run(["pkill", "-f", "uvicorn"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-f", "streamlit"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-f", "ngrok"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

CompletedProcess(args=['pkill', '-f', 'ngrok'], returncode=0)